In [1]:
from qiime2.plugins import fondue
from qiime2 import Artifact

import pandas as pd
import time

In [2]:
ids = Artifact.import_data('NCBIAccessionIDs', 'ids.tsv')
email = 'your@email.com'
n = 3

## Four jobs
Fetch 50 runs using 4 parallel jobs.

In [ ]:
times_many = []
n_jobs = 4

for i in range(n):
    t0 = time.time()
    results = fondue.methods.get_sequences(
        accession_ids=ids, 
        email=email,
        n_jobs=n_jobs,
        log_level='ERROR'
    )
    del results
    
    t1 = time.time()
    times_many.append(t1 - t0)
    time.sleep(600)

## One job
Fetch 50 runs using a single job.

In [ ]:
times_one = []
n_jobs = 1

for i in range(n):
    t0 = time.time()
    results = fondue.methods.get_sequences(
        accession_ids=ids, 
        email=email,
        n_jobs=n_jobs,
        log_level='ERROR'
    )
    del results
    
    t1 = time.time()
    times_one.append(t1 - t0)
    time.sleep(600)

## Evaluate results

In [ ]:
results_df = pd.DataFrame(data={'one_job': times_one, 'many_jobs': times_many})
results_df.to_csv('benchmark_result.csv', index=False)

In [ ]:
results_df.describe()